In [24]:
%pwd

'C:\\Users\\zeeshan.hameed\\Desktop\\assignment1\\assign1TSFD'

In [96]:
import os
os.chdir(r'C:\Users\zeeshan.hameed\Desktop\assignment1\assign1TSFD')

In [97]:
%pwd

'C:\\Users\\zeeshan.hameed\\Desktop\\assignment1\\assign1TSFD'

In [98]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareModel:
    model_direct : Path
    loss_type : str
    Batch_size: int
    epochs: int
    look_back : int


In [99]:
from timeseries.constants import *
from timeseries.utils.common import read_yaml,create_directories


class ConfigrationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    def prepare_model_config(self)->PrepareModel:
        config=self.config.prepare_model
        create_directories([config.model_directory])
        pre_model=PrepareModel(model_direct=Path(config.model_directory),loss_type=config.error_type,Batch_size=self.params.BATCH_SIZE,epochs=self.params.EPOCHS,
                               look_back=self.params.LOOK_BACK)
        return pre_model






In [104]:
import tensorflow as tf
from keras.models import Sequential
import keras
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.layers import Dense,Dropout
from keras.layers import LSTM

class TSmodel:
    def __init__(self,config :PrepareModel ):
        self.config = config
    
    def get_model(self):
        look_back = self.config.look_back
        loss_ty = self.config.loss_type
        model = Sequential()
        model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(look_back, 1)))
        model.add(LSTM(50, activation='relu', return_sequences=True))
        model.add(LSTM(50, activation='sigmoid', return_sequences=False))
        model.add(Dense(50))
        model.add(Dropout(0.2))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss=loss_ty)
        model.summary()
        return model
    
    def save_mod(self,model : keras.Model):
        os.makedirs(self.config.model_direct,exist_ok=True)
        path=self.config.model_direct
        model.save_weights(path/"lstm.h5")
        

In [105]:
config=ConfigrationManager()
prepare_model_config=config.prepare_model_config()
tsmodel=TSmodel(config=prepare_model_config)
modele=tsmodel.get_model()
a=tsmodel.save_mod(modele)

[2024-02-02 15:03:08,994: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-02 15:03:08,997: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-02 15:03:08,999: INFO: common: created directory at: artifacts]
[2024-02-02 15:03:09,001: INFO: common: created directory at: artifacts/prepare_model]
Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_63 (LSTM)              (None, 30, 50)            10400     
                                                                 
 lstm_64 (LSTM)              (None, 30, 50)            20200     
                                                                 
 lstm_65 (LSTM)              (None, 50)                20200     
                                                                 
 dense_42 (Dense)            (None, 50)                2550      
                                                